In [9]:
import folium
import pandas as pd
from math import radians, cos, sin, asin, sqrt, atan, log10,acos
import math

__base32 = '0123456789bcdefghjkmnpqrstuvwxyz'
x_pi = 3.14159265358979324 * 3000.0 / 180.0
pi = 3.1415926535897932384626  # π
a = 6378245.0  # 长半轴
ee = 0.00669342162296594323  # 扁率



def out_of_china(lng, lat):
    """
    判断是否在国内，不在国内不做偏移
    :param lng:
    :param lat:
    :return:
    """
    return not (lng > 73.66 and lng < 135.05 and lat > 3.86 and lat < 53.55)

def _transformlat(lng, lat):
    ret = -100.0 + 2.0 * lng + 3.0 * lat + 0.2 * lat * lat + \
          0.1 * lng * lat + 0.2 * math.sqrt(math.fabs(lng))
    ret += (20.0 * math.sin(6.0 * lng * pi) + 20.0 *
            math.sin(2.0 * lng * pi)) * 2.0 / 3.0
    ret += (20.0 * math.sin(lat * pi) + 40.0 *
            math.sin(lat / 3.0 * pi)) * 2.0 / 3.0
    ret += (160.0 * math.sin(lat / 12.0 * pi) + 320 *
            math.sin(lat * pi / 30.0)) * 2.0 / 3.0
    return ret


def _transformlng(lng, lat):
    ret = 300.0 + lng + 2.0 * lat + 0.1 * lng * lng + \
          0.1 * lng * lat + 0.1 * math.sqrt(math.fabs(lng))
    ret += (20.0 * math.sin(6.0 * lng * pi) + 20.0 *
            math.sin(2.0 * lng * pi)) * 2.0 / 3.0
    ret += (20.0 * math.sin(lng * pi) + 40.0 *
            math.sin(lng / 3.0 * pi)) * 2.0 / 3.0
    ret += (150.0 * math.sin(lng / 12.0 * pi) + 300.0 *
            math.sin(lng / 30.0 * pi)) * 2.0 / 3.0
    return ret

def bd09_to_gcj02(bd_lon, bd_lat):
    """
    百度坐标系(BD-09)转火星坐标系(GCJ-02)
    百度——>谷歌、高德
    :param bd_lat:百度坐标纬度
    :param bd_lon:百度坐标经度
    :return:转换后的坐标列表形式
    """
    x = bd_lon - 0.0065
    y = bd_lat - 0.006
    z = math.sqrt(x * x + y * y) - 0.00002 * math.sin(y * x_pi)
    theta = math.atan2(y, x) - 0.000003 * math.cos(x * x_pi)
    gg_lng = z * math.cos(theta)
    gg_lat = z * math.sin(theta)
    return [gg_lat,gg_lng]

#84转gc2
def wgs84_to_gcj02(lng, lat):
    """
    :return:
    """
    if out_of_china(lng, lat): 
        return lng, lat
    dlat = _transformlat(lng - 105.0, lat - 35.0)
    dlng = _transformlng(lng - 105.0, lat - 35.0)
    radlat = lat / 180.0 * pi
    magic = math.sin(radlat)
    magic = 1 - ee * magic * magic
    sqrtmagic = math.sqrt(magic)
    dlat = (dlat * 180.0) / ((a * (1 - ee)) / (magic * sqrtmagic) * pi)
    dlng = (dlng * 180.0) / (a / sqrtmagic * math.cos(radlat) * pi)
    mglat = lat + dlat
    mglng = lng + dlng
    return [mglat,mglng]

def gcj02_to_bd09(lng, lat):
    """
    ��������ϵ(GCJ-02)ת�ٶ�����ϵ(BD-09)
    �ȸ衢�ߵ¡���>�ٶ�
    :param lng:�������꾭��
    :param lat:��������γ��
    :return:
    """
    z = math.sqrt(lng * lng + lat * lat) + 0.00002 * math.sin(lat * x_pi)
    theta = math.atan2(lat, lng) + 0.000003 * math.cos(lng * x_pi)
    bd_lng = z * math.cos(theta) + 0.0065
    bd_lat = z * math.sin(theta) + 0.006
    return [bd_lng,bd_lat]

def wgs84_to_bd09(lon, lat):
    lon, lat = wgs84_to_gcj02(lon, lat)
    
    return gcj02_to_bd09(lon, lat)

#第一个原素为基站真实的位置
input_list=[
#[113.288361,22.490479],[113.288361,22.490479],[113.288361,22.490479],[113.288361,22.490479],[113.288361,22.490479],[113.288361,22.490479],[113.437012,22.247780],[113.437012,22.247780],[113.374313,22.335220],[113.350540,22.392160],[113.350540,22.392160],[113.350540,22.392160],[113.374298,22.335119],[113.276787,22.521917],[113.276787,22.521917],[113.276787,22.521917],[113.276787,22.521917],[113.375137,22.340420],[113.375137,22.340420],[113.276039,22.509991],[113.276039,22.509991],[113.276039,22.509991],[113.276039,22.509991],[113.328407,22.449711],[113.328407,22.449711],[113.328407,22.449711],[113.328407,22.449711],[113.344643,22.416115],[113.344643,22.416115],[113.344643,22.416115],[113.344643,22.416115],[113.344643,22.416115],[113.302818,22.487741],[113.302818,22.487741],[113.386482,22.340210],[113.280167,22.527210],[113.280167,22.527210],[113.280167,22.527210],[113.280167,22.527210],[113.277206,22.497950],[113.350777,22.383650],[113.342812,22.423429],[113.352074,22.386311],[113.352074,22.386311],[113.312149,22.485010],[113.312149,22.485010],[113.329758,22.440960],[113.340088,22.432121],[113.340088,22.432121],[113.340088,22.432121],[113.340088,22.432121],[113.351089,22.370171],[113.351089,22.370171],[113.282349,22.490450],[113.282349,22.490450],[113.282349,22.490450],[113.354012,22.355009],[113.354012,22.355009],[113.391563,22.340969],[113.350777,22.383650],[113.363182,22.348320],[113.342812,22.423429],[113.342812,22.423429],[113.342812,22.423429],[113.308662,22.483030],[113.351257,22.366880],[113.351257,22.366880],[113.351257,22.366880],[113.351257,22.366880],[113.351257,22.366880],[113.351257,22.366880],[113.325760,22.457649],[113.325760,22.457649]
[112.794037,23.168297],[112.792427,23.174385],[112.792427,23.174385],[112.786079,23.180729],[112.786079,23.180729],[112.786079,23.180729],[112.786079,23.180729],[112.786079,23.180729],[112.762581,23.192617],[112.762581,23.192617],[112.762581,23.192617],[112.725929,23.197090],[112.725929,23.197090],[112.725929,23.197090],[112.725929,23.197090],[112.725929,23.197090],[112.730522,23.196770],[112.730522,23.196770],[112.695961,23.205326],[112.695961,23.205326],[112.695961,23.205326],[112.695961,23.205326],[112.695961,23.205326],[112.695961,23.205326],[112.695961,23.205326],[112.695961,23.205326],[112.695961,23.205326],[112.662125,23.231159],[112.662125,23.231159],[112.662125,23.231159],[112.662125,23.231159],[112.662125,23.231159],[112.662125,23.231159],[112.662125,23.231159],[112.643944,23.200689],[112.643944,23.200689],[112.643944,23.200689],[112.643944,23.200689],[112.643944,23.200689],[112.643944,23.200689],[112.643944,23.200689],[112.643944,23.200689],[112.643944,23.200689],[112.643944,23.200689],[112.643944,23.200689],[112.643944,23.200689],[112.643944,23.200689],[112.643944,23.200689],[112.643944,23.200689],[112.643944,23.200689],[112.643944,23.200689],[112.643944,23.200689],[112.643944,23.200689],[112.643944,23.200689],[112.643944,23.200689],[112.643944,23.200689],[112.643944,23.200689],[112.643944,23.200689],[112.643944,23.200689],[112.643944,23.200689],[112.643944,23.200689],[112.643944,23.200689],[112.643944,23.200689],[112.643944,23.200689],[112.643944,23.200689],[112.643944,23.200689],[112.643944,23.200689],[112.643944,23.200689],[112.643944,23.200689],[112.662125,23.231159],[112.662125,23.231159],[112.662125,23.231159],[112.662125,23.231159],[112.662125,23.231159],[112.662125,23.231159],[112.654587,23.217916],[112.654587,23.217916],[112.654587,23.217916],[112.654587,23.217916],[112.622391,23.238770],[112.622391,23.238770],[112.617851,23.246527],[112.622391,23.238770],[112.617851,23.246527],[112.617851,23.246527],[112.617851,23.246527],[112.617851,23.246527],[112.617851,23.246527],[112.578873,23.325632],[112.578873,23.325632],[112.577553,23.327879],[112.578873,23.325632],[112.577553,23.327879],[112.577553,23.327879],[112.577553,23.327879],[112.558067,23.358120],[112.558067,23.358120],[112.558067,23.358120],[112.558067,23.358120],[112.504814,23.408682],[112.504814,23.408682],[112.504814,23.408682],[112.504814,23.408682],[112.504814,23.408682],[112.460121,23.461855],[112.460121,23.461855],[112.460121,23.461855],[112.444420,23.501646],[112.444420,23.501646],[112.444420,23.501646],[112.444420,23.501646],[112.444420,23.501646],[112.444420,23.501646],[112.434448,23.523350],[112.434448,23.523350],[112.434448,23.523350],[112.433113,23.543341],[112.433113,23.543341],[112.433113,23.543341],[112.431419,23.548359],[112.431419,23.548359],[112.431419,23.548359],[112.422035,23.587917],[112.422035,23.587917],[112.422035,23.587917],[112.421112,23.580374],[112.421112,23.580374],[112.418678,23.595133],[112.418678,23.595133],[112.418678,23.595133],[112.418678,23.595133],[112.422035,23.587917],[112.422035,23.587917],[112.422035,23.587917],[112.422035,23.587917],[112.422035,23.587917],[112.422035,23.587917],[112.422035,23.587917],[112.418571,23.595091],[112.418571,23.595091],[112.418571,23.595091],[112.418571,23.595091],[112.411186,23.620630],[112.411186,23.620630],[112.411186,23.620630],[112.411186,23.620630],[112.391991,23.640308],[112.391991,23.640308],[112.391991,23.640308],[112.391991,23.640308],[112.391991,23.640308],[112.359779,23.678497],[112.359779,23.678497],[112.359779,23.678497],[112.357506,23.681429],[112.359779,23.678497],[112.357506,23.681429],[112.359779,23.678497],[112.357506,23.681429],[112.350113,23.698015],[112.350113,23.698015],[112.350113,23.698015],[112.359779,23.678497],[112.350113,23.698015],[112.359779,23.678497],[112.345177,23.713514],[112.350113,23.698015],[112.350113,23.698015],[112.350113,23.698015],[112.350113,23.698015],[112.345177,23.713514],[112.345177,23.713514],[112.345177,23.713514],[112.315681,23.775909],[112.315681,23.775909],[112.315681,23.775909],[112.315681,23.775909],[112.315681,23.775909],[112.315681,23.775909],[112.315681,23.775909],[112.315681,23.775909],[112.315681,23.775909],[112.315681,23.775909],[112.313889,23.778109],[112.313889,23.778109],[112.313889,23.778109],[112.313889,23.778109],[112.292282,23.795528],[112.292282,23.795528],[112.271477,23.806883],[112.292282,23.795528],[112.271477,23.806883],[112.271477,23.806883],[112.239830,23.829420],[112.239830,23.829420],[112.239830,23.829420],[112.239830,23.829420],[112.239830,23.829420],[112.210815,23.851530],[112.239830,23.829420],[112.210815,23.851530],[112.210815,23.851530],[112.164207,23.884113],[112.164207,23.884113],[112.164207,23.884113],[112.153519,23.896420],[112.153519,23.896420],[112.153519,23.896420],[112.153519,23.896420],[112.026428,23.993357],[112.026428,23.993357],[112.026428,23.993357],[111.995201,24.024414],[111.995201,24.024414],[111.995201,24.024414]
        
]

gps_data=[
[113.54221013503141,22.23050198241694]    

]

more_2km_gps_data=[
[114.07213975694444,23.178399793836807],[114.06693522135417,23.183077799479168],[114.06693522135417,23.183077799479168],[114.06693522135417,23.183077799479168],[114.06069417317708,23.186818305121527],[114.06069417317708,23.186818305121527],[113.88018120659723,23.201205240885418],[113.7606751844618,23.203173828125],[113.90176784939236,23.1988232421875],[113.98646457248265,23.185299207899305],[113.98646457248265,23.185299207899305],[113.98646457248265,23.185299207899305],[113.90176784939236,23.1988232421875],[114.09462158203125,23.16422634548611],[114.09462158203125,23.16422634548611],[114.08734537760417,23.16614529079861],[114.08734537760417,23.16614529079861],[113.90176784939236,23.1988232421875],[113.90176784939236,23.1988232421875],[114.08734537760417,23.16614529079861],[114.08734537760417,23.16614529079861],[114.07687554253472,23.173304036458333],[114.08734537760417,23.16614529079861],[114.02482611762153,23.187647298177083],[114.02482611762153,23.187647298177083],[114.02482611762153,23.187647298177083],[114.01752549913195,23.186342502170138],[113.98646457248265,23.185299207899305],[113.98646457248265,23.185299207899305],[113.98646457248265,23.185299207899305],[113.98646457248265,23.185299207899305],[113.98646457248265,23.185299207899305],[114.06069417317708,23.186818305121527],[114.05398491753472,23.18925564236111],[113.88018120659723,23.201205240885418],[114.10964463975695,23.16244357638889],[114.10198513454861,23.16361083984375],[114.10198513454861,23.16361083984375],[114.10198513454861,23.16361083984375],[114.07687554253472,23.173304036458333],[113.98646457248265,23.185299207899305],[113.98646457248265,23.185299207899305],[113.98646457248265,23.185299207899305],[113.98646457248265,23.185299207899305],[114.04680718315973,23.19045627170139],[114.03940863715277,23.19030517578125],[114.0099232313368,23.185586480034722],[114.0099232313368,23.185586480034722],[114.001953125,23.18537814670139],[114.001953125,23.18537814670139],[114.001953125,23.18537814670139],[113.98646457248265,23.185299207899305],[113.98646457248265,23.185299207899305],[114.07687554253472,23.173304036458333],[114.07213975694444,23.178399793836807],[114.07213975694444,23.178399793836807],[113.91619249131945,23.19907497829861],[113.91619249131945,23.19907497829861],[113.90176784939236,23.1988232421875],[113.98646457248265,23.185299207899305],[113.91619249131945,23.19907497829861],[113.99456759982638,23.185272894965276],[113.99456759982638,23.185272894965276],[113.99456759982638,23.185272894965276],[114.04680718315973,23.19045627170139],[113.98646457248265,23.185299207899305],[113.98646457248265,23.185299207899305],[113.98646457248265,23.185299207899305],[113.98646457248265,23.185299207899305],[113.98646457248265,23.185299207899305],[113.98646457248265,23.185299207899305],[114.03189263237847,23.189001736111113],[114.02482611762153,23.187647298177083]
      
]    
    
    

reverse_latlng=[]

reverse_gps_latlng=[]

# #84转gc2
trans_list=[]
#84转bd
bd_list=[]

for lng_lat in input_list:
    trans_data=wgs84_to_gcj02(lng_lat[0], lng_lat[1])
    trans_list.append(trans_data)

# for lng_lat in input_list:
#     reverse_latlng.append([lng_lat[1],lng_lat[0]])

#84 to bd
# for lng_lat in input_list:
#     wgs84_to_bd09_data=wgs84_to_bd09(lng_lat[0], lng_lat[1])
# #     print(wgs84_to_bd09_data)
#     bd_list.append(wgs84_to_bd09_data)    

#小于2km的数据
for gps_lng_lat in gps_data:
    reverse_gps_latlng.append([gps_lng_lat[1],gps_lng_lat[0]])

#大于2km的数据
more_2km_list=[]    
for gps_lng_lat in more_2km_gps_data:
    more_2km_list.append([gps_lng_lat[1],gps_lng_lat[0]])

## 初始化地图，指定上海市
m = folium.Map(
    location=trans_list[0],
#     location=reverse_latlng[0],
    zoom_start=14,
    #tiles='http://webrd02.is.autonavi.com/appmaptile?lang=zh_cn&size=1&scale=1&style=7&x={x}&y={y}&z={z}', #老版
    tiles='http://wprd04.is.autonavi.com/appmaptile?lang=zh_cn&size=1&style=7&x={x}&y={y}&z={z}', #新版
    #tiles='https://wprd01.is.autonavi.com/appmaptile?x={x}&y={y}&z={z}&lang=zh_cn&size=1&scl=2&style=8&ltype=11', #路网

    attr='default'# 高德街道图
)


# 转移基站的位置,	113.49342777558105,22.811296556413257



# print(reverse_latlng)
#基站坐标
for loc in trans_list[0:]:
    folium.Marker(loc).add_to(m)

# print(bd_list)
#小于2km的坐标
for loc in reverse_gps_latlng[0:]:
    folium.Marker(loc,icon=folium.Icon(color="black")).add_to(m)    

#大于2km的坐标
for loc in more_2km_list[0:]:
    folium.Marker(loc,icon=folium.Icon(color="pink")).add_to(m)


m